In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import math
import random
plt.rcParams["animation.html"] = "jshtml"
%pylab inline
b = pd.read_csv("stolzenburg_marshall.csv")

In [ ]:
#Define Constants
n = 360
tmax = 4600
dt = 10 #seconds. why is there a dt here? doesn't make sense
e0 = 8.854187817e-12
asc_rate = .00444 
z0 = 7.2 #storm centered at 7 km altitude. need to adjust linspace so this works
dz = 100 # assuming storm is 100 km wide. this is wrong. i'm simultaneously assuming a finite and infinite width

#Set Altitude
z = linspace(0,16,n+1) #"n+1" creates a layer with width 1/n
threshold = 2.84e5*exp(-z[1:]/8) #2.84e5 is the base Electric Field required for Relativistic Electron Avalanche
#threshold presumably scales off of atmospheric density
#Define Functions
def f(z): #Function for Current. not quite sure what to do with it. it is proportional to this, however...
    #return -.3e-9*exp(-0.5*(z-4)**2) + 1.5e-9*exp(-.75*(z-7.5)**2) + -.5e-9*exp(-0.75*(z-13.75)**2)
    return 1e-7*exp(-(z-z0)**2/(2*dz**2))   #z0, dz probably not changed. this is also unitless.. needs fixing
#.00000 whatever is just so i can compare this with given data
def calcField(sigma):
    return A.dot(sigma) #Dot product between "A" and Charge Density. this seems to be ok for now

def discharge(sigma,E): #according to NSF proposal, lightning can be "crudely" modeled as a process that
                        #decreases sigma by a random fraction triggered randomly when E is above threshold
    for i in range(n):
        if any(abs(E[i]) >= threshold[i]):  # if any electric field is above the E required for lightning
            lightning_dis = 1e-9 #sigma[i] * random.uniform(.1, .9)  # lightning discharge is a certain fraction of sigma
            sigma[i] = sigma[i] - sign(sigma[i])*lightning_dis     # lightning discharge immediately removes a fraction of sigma
            # chance of lightning discharge should be random, but I want to focus on getting actual discharge working first
            if (n < 360):
                 sigma[i+1] = sigma[i+1] + sign(sigma[i+1])*1/2*lightning_dis
                 sigma[i-1] = sigma[i-1] + sign(sigma[i-1])*1/2*lightning_dis
            else:
                 sigma[i-1]=sigma[i-1] + sign(sigma[i-1])*lightning_dis
            E = calcField(sigma)   # E is recalculated
I = f(z)   # this seems ok

#Matrix used to calculate the Electric Field from the Charge Density
a = -tri(n, k=n) + tri(n) + tri(n,k=-1)   #not sure 3 tri functions are necessary... is one ok??
A = a*(1/(2*e0)) #Electric field of a sheet of charge (probably not fringe field)

In [ ]:
#Reset the time for every run through of the simulation
t = 0 #seconds

#Define Arrays and Lists
E = np.zeros(n) #Creates an array of "n" values... stays consistent with layer width
sigma = np.zeros(n) #Same reason as above
E_field = [] #Library for Electric Field profiles at each time step
sigma_array = [] #Library for Charge Density profiles at each time step

while t < tmax:
    sigma = sigma - diff(I)*dt

    #Electric Field
    E = calcField(sigma)
    E_field.append(copy(E))
        
    #For Lightning Discharges
    discharge(sigma,E)
                
    sigma_array.append(copy(sigma))
    t = t + dt

In [ ]:
#For data collection
t = 0
payloads = 1 #Number of data collections
delay = 1000
jj = 0
balloon_z = empty(payloads)

#Set Payload Release Times
release_time = [[] for i in range(payloads)]
for j in range(payloads):
    release_time[j].append(30*j + delay) #Release payloads every 30 seconds, starting after a specified delay

#Define Starting Altitude for Payloads
for j in range(payloads):
    balloon_z[j] = 0

stored_data = [[] for i in range(payloads)]
stored_height = [[] for i in range(payloads)]
stored_time = [[] for i in range(payloads)]

while t < tmax:
    balloon_data = empty((tmax-delay)//dt)
    for j in range(payloads):
        if t > release_time[j][0]:
            balloon_data[j] = np.interp(balloon_z[j],z[1:],E_field[jj])
            balloon_z[j] = balloon_z[j] + asc_rate*dt
            stored_data[j].append(balloon_data[j])
            stored_height[j].append(balloon_z[j])
            stored_time[j].append(t)
    jj = jj + 1  
    t = t + dt

In [ ]:
for j in range(payloads):
    plot(stored_data[j],stored_height[j], label="Payload " + str([j+1]))
plot(b['x'],b['y'],label="Literature Data")
plt.xlabel("E (V/m)")
plt.ylabel("Altitude (km)")
legend(loc="upper left")
figsize(8,8)

In [ ]:
for j in range(payloads):
    plot(stored_data[j],stored_time[j])
plt.xlabel("E (V/m)")
plt.ylabel("Time (s)")
figsize(8,8)

In [ ]:
fig = plt.figure()
ax = plt.axes(xlim=(-1e5,1e5), ylim=(0,16))
line, = ax.plot([], [], lw=2)
plot(b['x'],b['y'])
plot(threshold, z[1:])
plot(-threshold, z[1:])
plt.xlabel("E (V/m)")
plt.ylabel("Altitude (km)")
plt.close()

def init():
    line.set_data([], [])
    return line,
def animate(i):
    line.set_data(E_field[i],z[1:])
    return line,

animation.FuncAnimation(fig, animate, init_func=init, frames=360, interval=75, blit=True)

#Leaving this out for now so the notebook runs well